In [1]:
%run ../load_files.ipynb

In [2]:
%run ../data_dti_matrixes.ipynb

,profile,age,gender,MS_HV,MS_type,EDSS,disease_duration,education,connectivity,degree,...,MFIS,SF36_mental,SF36_physical,DTI_FA,DTI_L1,DTI_MD,DTI_RX,RAW,FUNC,LS
ID,,,,,,,,,,,,,,,,,,,,,
01_SLD,HV,35.090,female,HV,HV,NaN,NaN,NaN,****,30.026316,...,NaN,NaN,NaN,"[[0.0, 0.45681573943818, 0.0, 0.0, 0.0, 0.0, 0...","[[0.0, 0.00110047766107315, 0.0, 0.0, 0.0, 0.0...","[[0.0, 0.0007147709333892146, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0005219175716879154, 0.0, 0.0, 0.0, 0...","[[0.0, 0.03700267972016067, 0.0, 0.0, 0.0, 0.0...",NaN,NaN
02_MSG,HV,34.490,female,HV,HV,NaN,NaN,NaN,****,29.447368,...,NaN,NaN,NaN,"[[0.0, 0.4225060244392435, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001003761861901407, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0006781075016579274, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0005152803173683857, 0.0, 0.0, 0.0, 0...","[[0.0, 0.036053712695457325, 0.0, 0.0, 0.0, 0....",NaN,NaN
03_JAC,HV,34.351,male,HV,HV,NaN,NaN,NaN,****,30.078947,...,NaN,NaN,NaN,"[[0.0, 0.4472752719331546, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001120228893166676, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0007319733125098472, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0005378455096963582, 0.0, 0.0, 0.0, 0...","[[0.0, 0.020961570796962547, 0.0, 0.0, 0.0, 0....",NaN,NaN
04_NSV,HV,30.419,female,HV,HV,NaN,NaN,NaN,****,28.710526,...,NaN,NaN,NaN,"[[0.0, 0.4969576676177686, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001066288625096366, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0006679007988660829, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0004687068952403759, 0.0, 0.0, 0.0, 0...","[[0.0, 0.03931243595370451, 0.0, 0.0, 0.0, 0.0...",NaN,NaN
05_EGR,HV,42.030,female,HV,HV,NaN,NaN,NaN,****,30.526316,...,NaN,NaN,NaN,"[[0.0, 0.4787149021264542, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001099978842767666, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0006982670224039523, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0004974111118274082, 0.0, 0.0, 0.0, 0...","[[0.0, 0.030717934265789407, 0.0, 0.0, 0.0, 0....",NaN,NaN


(140, 39)


In [3]:
%run ../y_multilevel.ipynb

,attention,memory
ID,,
01_SLD,0,0
02_MSG,0,0
03_JAC,0,0
04_NSV,0,0
05_EGR,0,0


(140, 2)

In [4]:
y = y_values.memory

In [5]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.333, random_state=2018, stratify=y)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(93, 39)
(47, 39)
(93,)
(47,)


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    ('reduce_dim', PCA()),
    ('classify', GaussianNB())
])

In [8]:
# Parametrized values
N_FEATURES_OPTIONS = range(1, 500, 10)
MATRIXES = list(all_combinations(column_matrixes))
EXTRA_1D_COLUMS = [None] # + list(all_combinations(available_columns))                            

param_grid = { 
        'combining__columns': MATRIXES,
        'filter__columns': EXTRA_1D_COLUMS,
        
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
}


In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [10]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] reduce_dim__n_components=301, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD') 
[CV] reduce_dim__n_components=301, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD') 
[CV] reduce_dim__n_components=301, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD') 
[CV] reduce_dim__n_components=471, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX') 
[CV] reduce_dim__n_components=471, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX') 
[CV] reduce_dim__n_components=471, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX') 
[CV] reduce_dim__n_components=61, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD') 
[CV] reduce_dim__n_components=61, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD') 
[CV]  reduce_dim__n_components=301, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), score=0.59375, total=   2.8s
[CV] reduce_dim__n_componen

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   16.2s


[CV]  reduce_dim__n_components=211, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), score=0.7333333333333333, total=   3.1s
[CV] reduce_dim__n_components=101, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX') 
[CV]  reduce_dim__n_components=211, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), score=0.5483870967741935, total=   3.8s
[CV] reduce_dim__n_components=101, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX') 
[CV]  reduce_dim__n_components=131, filter__columns=None, combining__columns=('DTI_RX',), score=0.59375, total=   3.7s
[CV] reduce_dim__n_components=101, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX') 
[CV]  reduce_dim__n_components=131, filter__columns=None, combining__columns=('DTI_RX',), score=0.6666666666666666, total=   5.1s
[CV]  reduce_dim__n_components=131, filter__columns=None, combining__columns=('DTI_RX',), score=0.5483870967741935, total=   5.1s
[CV] reduce_dim__n_

[CV]  reduce_dim__n_components=41, filter__columns=None, combining__columns=('DTI_MD',), score=0.8, total=  12.9s
[CV] reduce_dim__n_components=81, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX') 
[CV]  reduce_dim__n_components=91, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), score=0.625, total=   2.1s
[CV]  reduce_dim__n_components=241, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), score=0.6451612903225806, total=   2.5s
[CV] reduce_dim__n_components=131, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX') 
[CV] reduce_dim__n_components=131, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX') 
[CV]  reduce_dim__n_components=241, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), score=0.6333333333333333, total=   2.4s
[CV]  reduce_dim__n_components=81, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), score=0.625, total=   2.4s
[CV] reduce_dim__n_compon

[CV]  reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1'), score=0.8333333333333334, total=   2.7s
[CV] reduce_dim__n_components=481, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD') 
[CV] reduce_dim__n_components=481, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD') 
[CV]  reduce_dim__n_components=291, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), score=0.8333333333333334, total=   2.5s
[CV]  reduce_dim__n_components=291, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), score=0.625, total=   2.9s
[CV] reduce_dim__n_components=481, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD') 
[CV] reduce_dim__n_components=311, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX') 
[CV]  reduce_dim__n_components=291, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), score=0.6451612903225806, total=   3.1s
[CV] reduce_dim__n_components=311, filter__columns=None, combinin

...

[CV]  reduce_dim__n_components=411, filter__columns=None, combining__columns=('DTI_MD',), score=0.6129032258064516, total=   1.5s
[CV] reduce_dim__n_components=151, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD') 
[CV]  reduce_dim__n_components=411, filter__columns=None, combining__columns=('DTI_MD',), score=0.7, total=   1.5s
[CV] reduce_dim__n_components=371, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD') 
[CV] reduce_dim__n_components=371, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD') 
[CV]  reduce_dim__n_components=21, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD'), score=0.65625, total=   2.6s
[CV] reduce_dim__n_components=371, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD') 
[CV]  reduce_dim__n_components=21, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD'), score=0.6451612903225806, total=   3.1s
[CV]  reduce_dim__n_components=21, filter__colum

[CV]  reduce_dim__n_components=51, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), score=0.3870967741935484, total=   1.6s
[CV] reduce_dim__n_components=371, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX') 
[CV]  reduce_dim__n_components=401, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), score=0.59375, total=   1.4s
[CV] reduce_dim__n_components=371, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX') 
[CV]  reduce_dim__n_components=51, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), score=0.8333333333333334, total=   2.6s
[CV] reduce_dim__n_components=371, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX') 
[CV]  reduce_dim__n_components=21, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), score=0.625, total=   3.1s
[CV]  reduce_dim__n_components=401, filter__columns=None, combining__columns=('DTI_L1',

[CV]  reduce_dim__n_components=381, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), score=0.625, total=   1.3s
[CV] reduce_dim__n_components=201, filter__columns=None, combining__columns=('DTI_MD',) 
[CV]  reduce_dim__n_components=381, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), score=0.6129032258064516, total=   1.6s
[CV] reduce_dim__n_components=421, filter__columns=None, combining__columns=('DTI_L1',) 
[CV]  reduce_dim__n_components=61, filter__columns=None, combining__columns=('DTI_MD',), score=0.5625, total=   1.3s
[CV]  reduce_dim__n_components=381, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), score=0.8333333333333334, total=   1.6s
[CV] reduce_dim__n_components=421, filter__columns=None, combining__columns=('DTI_L1',) 
[CV]  reduce_dim__n_components=61, filter__columns=None, combining__columns=('DTI_MD',), score=0.6129032258064516, total=   1.5s
[CV] reduce_dim__n_components=421, filter__columns=No

[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  3.8min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
          error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('combining', CombineMatrixTransformer(column_name='combined', columns=None,
             op=<function mean at 0x7f768c243840>)), ('spread_out', SpreadOutMatrixTransformer(columns=['combined'], symmetric=True)), ('filter', FeatureMatrixTransformer(columns=...om_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classify', GaussianNB(priors=None))]),
          fit_params=None, iid=True, n_iter=150, n_jobs=-1,
          param_distributions={'combining__columns': [('DTI_FA',), ('DTI_L1',), ('DTI_MD',), ('DTI_RX',), ('DTI_FA', 'DTI_L1'), ('DTI_FA', 'DTI_MD'), ('DTI_FA', 'DTI_RX'), ('DTI_L1', 'DTI_MD'), ('DTI_L1', 'DTI_RX'), ('DTI_MD', 'DTI_RX'), ('DTI_FA', 'DTI_L1', 'DTI_MD'), ('DTI_FA', 'DTI_L1', 'DTI_RX'), ('DTI_FA', 'DTI_MD', 'DTI_RX'), ('DTI_L1', 'DTI_MD', 'DTI_RX'), ('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX')], '

In [11]:
%run ../grid_results.ipynb

reduce_dim__n_components = 11
filter__columns = None
combining__columns = ('DTI_L1',)
Test
Accuracy in test: 78.72%
             precision    recall  f1-score   support

          0       0.78      1.00      0.88        35
          1       1.00      0.17      0.29        12

avg / total       0.83      0.79      0.72        47

Train
Accuracy in train: 86.02%
             precision    recall  f1-score   support

          0       0.87      0.96      0.91        70
          1       0.81      0.57      0.67        23

avg / total       0.86      0.86      0.85        93

